In [1]:
import os

import pandas as pd

import numpy as np

import statistics as stat

import matplotlib

import glob

In [2]:
from pathlib import Path

path = r'C:\projects\CONUS_TGW_WRF_Historical\CONUS_TGW_WRF_Historical'

files = Path(path).glob('*.csv')

In [3]:
dfs = list()
for f in files:
    data = pd.read_csv(f)
    # .stem is method for pathlib objects to get the filename w/o the extension
    data['file'] = f.stem
    dfs.append(data)
    
plant_df = pd.concat(dfs, ignore_index=True)

print(plant_df)